In [1]:
! pip install pickle-mixin

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# 워닝 은닉
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
# 라이브러리 import
import pickle
import pandas as pd
import numpy as np
import re

In [4]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# columns 생략 없이 출력
pd.set_option('display.max_columns', None)
# 문자열 출력 길이
pd.options.display.max_colwidth = 40

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
# row 데이터 불러오기
with open('/content/drive/MyDrive/CP2/df_secured3.pickle', 'rb') as f:
    data = pickle.load(f)

In [7]:
# 원본 카피
df = data.copy()

In [8]:
# read_receipt_secured 삭제
df.drop(columns=["read_receipt_secured"], inplace=True)

# 전송 날짜 타입 변경 object -> datetime
df['created_at'] = pd.to_datetime(df['created_at'])

# 닉 + 아이디
# df["nick_id"] = df["sender_nickname_secured"] + " / " + df["sender_id_secured"]

# 입퇴장 시 자동 생성 메세지 행 삭제
join = df[df['message'].str.contains('님이 참여했습니다.')].index
df.drop(join, inplace =True)
out = df[df['message'].str.contains('님이 나갔습니다.')].index
df.drop(out, inplace =True)

# 스티커명 컬럼 생성
sticker_name1 = df[df["type"]=="FILE"]["message"].str.split('_').str[1]
sticker_name2 = df[df["type"]=="FILE"]["message"].str.split('_').str[2]
sticker_name3 = df[df["type"]=="FILE"]["message"].str.split('_').str[3]
sticker_name2.fillna("_", inplace=True)
sticker_name3.fillna("_", inplace=True)
df["sticker_name"] = sticker_name1 + sticker_name2 + sticker_name3

# 후원금액 컬럼 생성
def sticker_price(x):
    if x == "Lollipop.png__":
        return 1
    elif x == "Heart.png__":
        return 10
    elif x == "I’mhere!.png__":
        return 20
    elif x == "Kiss.png__":
        return 33
    elif x == "CupofCoffee.png__":
        return 35
    elif x == "goodnight.png__":
        return 50
    elif x == "Makeitrain.png__":
        return 50
    elif x == "Hereforyou.png__":
        return 100
    elif x == "Abouttoloseit.png__":
        return 100
    elif x == "Overthemoon.png__":
        return 100
    elif x == "Partytime.png__":
        return 100
    elif x == "Flowerbouquet.png__":
        return 100
    elif x == "Chickendinner.png__":
        return 200
    elif x == "Congrats!.png__":
        return 300
    elif x == "Cake.png__":
        return 300
    elif x == "Fruitbox.png__":
        return 500
    elif x == "OH~.png__":
        return 500
    elif x == "Sneakers.png__":
        return 800
    elif x == "Go!Go!Go!.png__":
        return 1000
    elif x == "Diamond.png__":
        return 1000
    elif x == "HappyBirthday.png__":
        return 3000
    elif x == "Let’splay.png__":
        return 5000
    elif x == "HolidayVibes.png__":
        return 10000
    elif x == "giftset1.png":
        return 100
    elif x == "giftset2.png":
        return 500
    elif x == "giftset3.png":
        return 1000
    elif x == "wi120220707.png":
        return 100
    elif x == "02위칭.png_":
        return 100
    elif x == "bi120220707.png":
        return 100
    elif x == "나윤님추천수정.png":
        return 100
    elif x == "나윤님비추.png_":
        return 100
    elif x == "라마1004님배경없음.png_":
        return 10
    elif x == "bom120220707.png":
        return 100
    elif x == "so120220707":
        return 100
    elif x == "해빈1.png_":
        return 30
    elif x == "해빈2.png_":
        return 30
    elif x == "해빈3.png_":
        return 30
    elif x == "백찬1.png_":
        return 30
    elif x == "백찬2.png_":
        return 30
    elif x == "백찬3.png_":
        return 30
    elif x == "이안1.png_":
        return 30
    elif x == "이안2.png_":
        return 30
    elif x == "이안3.png_":
        return 30
    elif x == "윤1.png_":
        return 30
    elif x == "윤2.png_":
        return 30
    elif x == "윤3.png_":
        return 30
    elif x == "호림1.png_":
        return 30
    elif x == "호림2.png_":
        return 30
    elif x == "호림3.png_":
        return 30
    elif x == "영보1.png__":
        return 30
    elif x == "영보3.png__":
        return 30
    elif x == "YG미쯔.jpg_":
        return 100
    elif x == "YG천사.jpg_":
        return 1004
    elif x == "nb1.jpeg_":
        return 20
    elif x == "KLian3.png_":
        return 10
    elif x == "ma120220707.png":
        return 100
    elif x == "q220220707.png":
        return 100
    elif x == "q520220707":
        return 100
    elif x == "q320220707":
        return 100
    elif x == "YH120220724033317.png":
        return 10
    else:
        return 0
df["sticker_price"] = df["sticker_name"].apply(lambda x : sticker_price(x))

In [9]:
# 개인 커뮤니티 여부
df["private_community"] = 0

tgd_index = df[df['커뮤니티 링크'].str.contains('tgd', na=False)].index
discord_index = df[df['커뮤니티 링크'].str.contains('discord', na=False)].index
cafe_index = df[df['커뮤니티 링크'].str.contains('cafe', na=False)].index
etc_index1 = df[df['커뮤니티 링크'].str.contains('트게더', na=False)].index
etc_index2 = df[df['커뮤니티 링크'].str.contains('카페', na=False)].index
etc_index3 = df[df['커뮤니티 링크'].str.contains('디스코드', na=False)].index
etc_index4 = df[df['커뮤니티 링크'].str.contains('트개더', na=False)].index

df.loc[tgd_index, ['private_community']] = 1
df.loc[discord_index, ['private_community']] = 1
df.loc[cafe_index, ['private_community']] = 1
df.loc[etc_index1, ['private_community']] = 1
df.loc[etc_index2, ['private_community']] = 1
df.loc[etc_index3, ['private_community']] = 1
df.loc[etc_index4, ['private_community']] = 1

In [10]:
# 메세지 길이
df['message_len'] = df['message'].str.len()

In [11]:
# 아프리카 컬럼 결측치 제거 및 데이터 단일화(?) 그리고 자료형 인트화
# 트위치의 팔로워 개념과 같은 애청자(즐겨찾기)만 남겨놓음

df['아프리카(애청자/팬클럽)'].fillna(0, inplace = True)
df['아프리카(애청자/팬클럽)'] = df['아프리카(애청자/팬클럽)'].replace('15,777/11542', '15777')
df['아프리카(애청자/팬클럽)'] = df['아프리카(애청자/팬클럽)'].replace('13,763/2430', '13763')
df['아프리카(애청자/팬클럽)'] = df['아프리카(애청자/팬클럽)'].replace('171,372/18,901', '171372')
df['아프리카(애청자/팬클럽)'] = df['아프리카(애청자/팬클럽)'].replace('9,069/1,848', '9069')
df['아프리카(애청자/팬클럽)'] = df['아프리카(애청자/팬클럽)'].replace('1,600/1,135', '1600')
df['아프리카(애청자/팬클럽)'] = df['아프리카(애청자/팬클럽)'].replace('4.7천', '4700')
df['아프리카(애청자/팬클럽)'] = df['아프리카(애청자/팬클럽)'].replace('2,663/2,659', '2663')
df['아프리카(애청자/팬클럽)'] = df['아프리카(애청자/팬클럽)'].replace('1.9천', '1900')
df = df.astype({'아프리카(애청자/팬클럽)':'int'})

df['유튜브 수'].fillna(0, inplace = True)
df['유튜브 수'] = df['유튜브 수'].replace('?', '0')

In [12]:
# 채널주인 url은 다르나 닉네임이 같은 경우 구분 ex) 마녀** -> 마녀**_1, 마녀**_2
idx_1 = df[(df["채널주인"] == "sendbird_group_channel_85027934_96d14caeb215fa030b76ab8a449dafae0e252bfe") & (df["sender_nickname_secured"] == "마녀**")].index
idx_2 = df[(df["채널주인"] == "sendbird_group_channel_49408541_0366b6c9039293477356a8ab37fd7317d6a25228") & (df["sender_nickname_secured"] == "마녀**")].index
idx_3 = df[(df["채널주인"] == "sendbird_group_channel_51673049_e9127466665f7535749033f986c3e01c7a67cfc1") & (df["sender_nickname_secured"] == "🍒체******")].index
idx_4 = df[(df["채널주인"] == "sendbird_group_channel_88950504_2e806ea8bf0ab44eaa5fb93978529427c2053262") & (df["sender_nickname_secured"] == "🍒체******")].index
idx_5 = df[(df["채널주인"] == "sendbird_group_channel_82333993_7fe5b9c11092fdc107a23a6dcb482634eb75b3da") & (df["sender_nickname_secured"] == "🍒체******")].index
idx_6 = df[(df["채널주인"] == "sendbird_group_channel_82336139_dc97c0b3b177072e91bf628625d5fa7539db532a") & (df["sender_nickname_secured"] == "🍒체******")].index
idx_7 = df[(df["채널주인"] == "sendbird_group_channel_51672917_5e13f3722cbeec5c766726c332c063e310e5f9ce") & (df["sender_nickname_secured"] == "🍒체******")].index
idx_8 = df[(df["채널주인"] == "sendbird_group_channel_82337532_c81ea37759a4761f4f6989e5f0f170f2e7a723fd") & (df["sender_nickname_secured"] == "🍒체******")].index

df.loc[idx_1, ['sender_nickname_secured']] = "마녀**_1"
df.loc[idx_2, ['sender_nickname_secured']] = "마녀**_2"
df.loc[idx_3, ['sender_nickname_secured']] = "🍒체******_1"
df.loc[idx_4, ['sender_nickname_secured']] = "🍒체******_2"
df.loc[idx_5, ['sender_nickname_secured']] = "🍒체******_3"
df.loc[idx_6, ['sender_nickname_secured']] = "🍒체******_4"
df.loc[idx_7, ['sender_nickname_secured']] = "🍒체******_5"
df.loc[idx_8, ['sender_nickname_secured']] = "🍒체******_6"

In [13]:
# 채널주인 url은 다르나 채널이름이 같은 경우 구분
owner_url1 = df[df["sender_nickname_secured"] == "🍒체******_2"]["채널주인"].unique()
owner_url2 = df[df["sender_nickname_secured"] == "🍒체******_3"]["채널주인"].unique()
owner_url3 = df[df["sender_nickname_secured"] == "🍒체******_4"]["채널주인"].unique()

owner_url4 = df[df["sender_nickname_secured"] == "🍒체******_1"]["채널주인"].unique()
owner_url5 = df[df["sender_nickname_secured"] == "🍒체******_5"]["채널주인"].unique()

idx_9 = df[df["channel_url_x"] == owner_url1[0]].index
idx_10 = df[df["channel_url_x"] == owner_url2[0]].index
idx_11 = df[df["channel_url_x"] == owner_url3[0]].index

idx_12 = df[df["channel_url_x"] == owner_url4[0]].index
idx_13 = df[df["channel_url_x"] == owner_url5[0]].index

df.loc[idx_9, ['channel_name_secured']] = "🍒체******_1"
df.loc[idx_10, ['channel_name_secured']] = "🍒체******_2"
df.loc[idx_11, ['channel_name_secured']] = "🍒체******_3"

df.loc[idx_12, ['channel_name_secured']] = "🍒체********************_1"
df.loc[idx_13, ['channel_name_secured']] = "🍒체********************_2"

In [15]:
# 피클 파일 저장 df_clean.pickle
with open('/content/drive/MyDrive/CP2/df_clean3.pickle', 'wb') as f:
	pickle.dump(df, f, protocol=pickle.HIGHEST_PROTOCOL)